In [ ]:
import os
import json
import time
import re
from datetime import datetime
from typing import Any, Dict, List, Tuple

# CHANGED: Support both Hugging Face and Google Gemini
from huggingface_hub import InferenceClient
try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    
import validator

# =========================================================
# 0. Prompt Definitions
# =========================================================
BASE_CHOREOGRAPHY_GEN_PROMPT = """
You are a choreographer AI. Generate a dance choreography for 2–5 dancers. 
Write it in natural English, describing entrances, movements, locations on stage (left, right, center, up-left, up-right, etc.) and give approximate times (in seconds). 
Keep it around 60–90 seconds.
"""


CHOREOGRAPHY_GEN_PROMPT = """
You are an expert Choreography Design System. Your goal is to create a conceptually rich and spatially complex dance piece for 2–5 dancers.

To achieve high-quality output, follow this process:

1. **Meta-Analysis & Setup:**
   - define a specific dance style (e.g., Contemporary, Jazz, Ballet).
   - Define the "Mood" and "Narrative Arc" of the piece.
   - Assign specific roles or character dynamics to the dancers (e.g., Dancers A & B are mirrored, Dancer C is the disruptor).
   - think step by step of the structure of the 60–90 seconds.
   - Break it down into distinct sections
   - Plan the spatial usage: Ensure dancers use the full stage.

3. **Final Output Generation:**
   - Based on the planning above, generate the final choreography script in natural English.
   - Include precise timestamps
   - Explicitly describe when each dancer **enters** and **exits** the stage.
   - Movements should be physically possible (dancers cannot teleport across the stage instantly).
   - Describe movements, specific stage locations, and interactions between dancers clearly.

"""

JSON_CONVERSION_TEMPLATE = """
Convert the choreography text into structured JSON. 

Rules: 
- Represent the stage as a 25x25 grid: x in [0..24], y in [0..24] 
- Include only events with fields: id, dancer, action, start, end, from, to, partner (optional) 
- Use seconds for start/end (float allowed) 
- If a movement has no clear start or end, infer them reasonably 
- Map narrative locations to grid coordinates: 
    "left"=0, "right"=24, "center"=12, "up"=0, "down"=24 
    up-left=(0,0), up-right=(24,0), down-left=(0,24), down-right=(24,24)
- **from** and **to** must be represented as dictionaries with x and y keys, for example: 
      "from": {"x": 0, "y": 2},
      "to": {"x": 1, "y": 2}
- If an action is for all dancers, repeat it for each dancer individually.

- **IMPORTANT:** 
    1. Every dancer MUST have an explicit "enter" event (first event) and "exit" event (last event).
    2. Dancers cannot move more than 4 cells in 1 second (speed limit).
    3. Dancers cannot occupy the same (x,y) at the same time (collision).
    4. Dancers cannot pause for more than 60 seconds.

Return ONLY valid JSON in this format: 
{ 
  "dancers": [...], 
  "duration": ..., 
  "events": [...] 
}

Choreography text: <<<TEXT_HERE>>>
"""


# =========================================================
# 1. Filesystem helpers
# =========================================================

def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def save_text(path: str, text: str) -> None:
    ensure_dir(os.path.dirname(path))
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)


def save_json(path: str, data: Any) -> None:
    ensure_dir(os.path.dirname(path))
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


def make_run_folder(base_dir: str, model_name: str, run_id: str) -> str:
    safe_model_name = model_name.replace("/", "-")
    run_dir = os.path.join(base_dir, "logs", safe_model_name, run_id)
    ensure_dir(run_dir)
    return run_dir


# =========================================================
# 2. Model Detection & Client Initialization
# =========================================================

def is_gemini_model(model_name: str) -> bool:
    """Check if the model is a Gemini model."""
    return "gemini" in model_name.lower()


def init_client(api_key: str, model_name: str):
    """
    Initialize the appropriate client based on model type.
    Returns (client, client_type) where client_type is 'gemini' or 'huggingface'
    """
    if not api_key:
        raise ValueError("API key is required.")
    
    if is_gemini_model(model_name):
        if not GEMINI_AVAILABLE:
            raise ImportError("google-generativeai package not installed. Install with: pip install google-generativeai")
        genai.configure(api_key=api_key)
        return genai.GenerativeModel(model_name), 'gemini'
    else:
        return InferenceClient(token=api_key), 'huggingface'


# =========================================================
# 3. Unified API Call Functions
# =========================================================

from typing import Tuple, Optional
import time

def call_model(client, prompt: str, model_name: str, client_type: str, temperature: float = 0.7) -> str:
    """
    Make a completion call using the appropriate client.
    """
    if client_type == 'gemini':
        # Gemini requires temperature to be passed in generation_config
        response = client.generate_content(
            prompt,
            generation_config={"temperature": temperature}
        )
        return response.text
    else:  # huggingface
        response = client.chat_completion(
            model=model_name,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=6000, 
            temperature=temperature 
        )
        return response.choices[0].message.content


def call_model_with_retry(
    client,
    prompt: str,
    model_name: str,
    client_type: str,
    temperature: float = 1,  # Added temperature parameter here
    max_retries: int = 5,
    initial_backoff: float = 2.0,
) -> Tuple[Optional[str], Optional[str]]:
    """
    Call model with retry and exponential backoff.
    Works with both Gemini and Hugging Face.
    """
    backoff = initial_backoff
    last_error = None

    for attempt in range(1, max_retries + 1):
        try:
            # Pass the temperature to the inner function
            text = call_model(client, prompt, model_name, client_type, temperature=temperature)
            return text, None
        except Exception as e:
            last_error = f"Attempt {attempt} failed: {e}"
            print(last_error)
            
            # Handle model loading or rate limits
            error_str = str(e).lower()
            if "loading" in error_str or "503" in error_str or "quota" in error_str:
                print("   (Model is loading or rate limited... waiting longer)")
                time.sleep(backoff * 2)
            
            if attempt < max_retries:
                time.sleep(backoff)
                backoff *= 2

    return None, last_error

# =========================================================
# 4. JSON extraction + safe parsing
# =========================================================

def extract_json(text: str) -> str:
    """
    Extract the FIRST complete top-level JSON object using brace matching.
    Handles truncation by returning the longest balanced substring.
    """
    start = text.find("{")
    if start == -1:
        return ""

    depth = 0
    end = None

    for i in range(start, len(text)):
        if text[i] == "{":
            depth += 1
        elif text[i] == "}":
            depth -= 1
            if depth == 0:
                end = i
                break

    if end is None:
        # JSON is truncated — return partial for repair
        return text[start:].strip()

    return text[start:end+1].strip()


def is_truncated(json_str: str) -> bool:
    """
    Detect if JSON is truncated by checking brace balance.
    """
    return json_str.count("{") != json_str.count("}")


def parse_json_safe(text: str) -> Tuple[Dict | None, str | None]:
    cleaned = extract_json(text)

    if is_truncated(cleaned):
        return None, "JSON appears truncated (unbalanced braces)."

    try:
        parsed = json.loads(cleaned)
        return parsed, None
    except json.JSONDecodeError as e:
        return None, f"JSON parsing error at line {e.lineno}: {e.msg}"


# =========================================================
# 5. Generation functions
# =========================================================

def generate_choreography_text(client, model_name: str, client_type: str, temperature: float = 1) -> Tuple[str | None, str | None]:
    text, err = call_model_with_retry(client=client, prompt=CHOREOGRAPHY_GEN_PROMPT, 
                                      model_name=model_name, client_type=client_type, temperature = temperature)
    return text, err


def generate_choreography_json(
    text_choreography: str,
    client,
    model_name: str,
    client_type: str, 
    temperature: float = 1
) -> Tuple[Dict | None, str | None, str | None]:

    prompt = JSON_CONVERSION_TEMPLATE.replace("<<<TEXT_HERE>>>", text_choreography)

    # First attempt
    raw_json, err = call_model_with_retry(client=client, prompt=prompt, 
                                          model_name=model_name, client_type=client_type, temperature = temperature)
    if err or raw_json is None:
        return None, None, err
    cleaned_json_str = extract_json(raw_json)
    parsed, parse_err = parse_json_safe(cleaned_json_str)
    return parsed, cleaned_json_str, parse_err


# =========================================================
# 6. Validation & Refinement Logic
# =========================================================

def generate_feedback_prompt(violations: List[Dict]) -> str:
    violation_summary = json.dumps(violations, indent=2)

    return f"""
The following choreography JSON contains validation errors. Fix them.

Errors:
{violation_summary}

**Refinement Requirements:**
- Correct every violation listed above.
- Ensure all dancers have explicit 'enter' and 'exit' actions.
- enteries and exits should be seperated from the rest of the dance they are place holders to track please pay attention.
- Ensure no two dancers share the same (x,y) at the same time.
- Ensure no dancer stays still for more than 60 seconds.
- Keep all coordinates within the 0–24 stage boundaries.
- Maintain chronological consistency.
- Output ONLY valid JSON. No explanations, no markdown.
- Ensure movement speed never exceeds 4 grid cells per second.

Regenerate the full corrected JSON now.
"""


def refine_choreography_loop(client, model_name: str, client_type: str, initial_json: Dict, 
                             run_dir: str, max_iterations: int = 10, temperature: float = 1) -> Dict:
    current_json = initial_json
    save_json(os.path.join(run_dir, "iteration_0.json"), current_json)

    for i in range(1, max_iterations + 1):
        print(f"--- Validating Iteration {i-1} ---")
        report = validator.validate(current_json)
        save_json(os.path.join(run_dir, f"report_{i-1}.json"), report)

        if report["valid"]:
            print(f"✅ SUCCESS! Valid choreography found at iteration {i-1}")
            save_json(os.path.join(run_dir, "final_valid.json"), current_json)
            return {"success": True, "iterations": i-1, "final_json": current_json}
        
        print(f"❌ Iteration {i-1} Failed. Violations: {len(report['violations'])}")
        feedback_prompt = generate_feedback_prompt(report["violations"])
        
        full_refinement_prompt = f"Current JSON:\n{json.dumps(current_json)}\n\n{feedback_prompt}"
        
        print(f"--- Requesting Fix (Iteration {i}) ---")
        raw_text, err = call_model_with_retry(client, full_refinement_prompt, 
                                              model_name, client_type, temperature = temperature)
        
        if err:
            print(f"API Error during refinement: {err}")
            break
            
        new_json_str = extract_json(raw_text)
        parsed, parse_err = parse_json_safe(new_json_str)
        
        if parse_err:
            print(f"JSON Parsing failed at iteration {i}: {parse_err}")
            save_text(os.path.join(run_dir, f"failed_json_iter_{i}.txt"), raw_text)
            continue
            
        current_json = parsed
        save_json(os.path.join(run_dir, f"iteration_{i}.json"), current_json)

    print("❌ Reached maximum iterations without success.")
    return {"success": False, "iterations": max_iterations, "final_json": current_json}


# =========================================================
# 7. Main Pipeline
# =========================================================

def run_pipeline(api_key: str, model_name: str):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_dir = "modified_choreography_prompt"
    temperature = 0.1
    temp_str = f"temp_{temperature}"
    run_dir = make_run_folder(base_dir, model_name, f"run_{temp_str}_{timestamp}")
    
    print(f"🚀 Starting Run: {timestamp}")
    print(f"📂 Output Directory: {run_dir}")
    print(f"🤖 Model: {model_name}")
    
    # Initialize the appropriate client
    client, client_type = init_client(api_key, model_name)
    print(f"🔧 Using {client_type} client")
    # --- Step 1 ---
    print("\n[1/3] Generating Natural Language Choreography...")
    text_choreo, err = generate_choreography_text(client, model_name, client_type, temperature)
    if err:
        print(f"Critical Error: {err}")
        return
    print("   -> Text generated.")
    save_text(os.path.join(run_dir, "choreography.txt"), text_choreo)

    # --- Step 2 ---
    print("\n[2/3] Converting to JSON...")
    initial_json, raw_json_str, parse_err = generate_choreography_json(
        text_choreo, client, model_name, client_type, temperature)
    if parse_err:
        print(f"Critical JSON Error: {parse_err}")
        save_text(os.path.join(run_dir, "failed_initial_json.txt"), raw_json_str)
        return
    print("   -> Initial JSON created.")

    # --- Step 3 ---
    print("\n[3/3] Entering Validation Loop...")
    result = refine_choreography_loop(client, model_name, client_type, initial_json, run_dir, max_iterations = 6, temperature = temperature)

    # --- Summary ---
    summary = {
        "model": model_name,
        "timestamp": timestamp,
        "status": "Success" if result["success"] else "Failed",
        "iterations_required": result["iterations"],
        "client_type": client_type,
        "temperature": temperature,
    }
    save_json(os.path.join(run_dir, "summary.json"), summary)
    print("\nDone.")


###########################

API_KEY = "gemini_API_KEY"

MODEL_NAME = "gemini-2.5-flash-lite"
MODEL_NAME = "gemini-2.5-flash"
MODEL_NAME = "gemini-3-pro-preview"


run_pipeline(API_KEY, MODEL_NAME)